In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import requests
import openpyxl
import os
import string
import nltk
import syllables
import re


In [ ]:
df = pd.read_csv('Input.csv')
# contains URLS of wesbites and also the URL_NO

In [ ]:
df.head()

In [ ]:
# loop through the URLs and save all data to files
for url_id, url in zip(df['URL_ID'], df['URL']):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    div_tag = soup.find('div', class_='td-post-content')

    if div_tag != None:
        data1 = div_tag.find_all(text=True)
        data = ''
        data = soup.find('title').text[:-23]

        for text in data1:
            data += (text)

        f = open(str(url_id)+".txt", "w+", encoding="utf-8")
        n = f.write(data)
        f.close()

    else:
        f = open(str(url_id)+".txt", "w+", encoding="utf-8")
        n = f.write("")
        f.close()


In [ ]:
# check for stop words in a folder
# you can also use stop words from nltk library
folder_path = 'StopWords'
word_list = []

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)

    if os.path.isfile(file_path):
        with open(file_path, 'r') as f:
            text = f.read()

        words = text.split()
        word_list.extend(words)
f.close()


In [ ]:
# save output in the CSV file
df_output = pd.read_csv('Output Data Structure.csv')
print(df_output)

In [ ]:
i = 0

positive_words = []

with open(r"enter you path for positve word files", 'r', encoding="utf-8") as f:
    text = f.read()

f.close()
positive = text.split()
positive_words.extend(positive)

# print(positive_words)

negative_words = []

with open(r"enter you path for negative word files", 'r') as f:
    text = f.read()

f.close()
negative = text.split()
negative_words.extend(negative)

# print(negative_words)

for url_id, url in zip(df_output['URL_ID'], df_output['URL']):
    with open(str(url_id)+".txt", 'r+', encoding="utf-8") as f:
        text = f.read()

    words = nltk.word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in word_list]
    positive_score = 0
    negative_score = 0
    for words in filtered_words:
        if words in positive_words:
            positive_score += 1
        elif words in negative_words:
            negative_score += -1

    # calculate negative and it should be positive so *-1
    negative_score *= -1

    sentences = nltk.sent_tokenize(text)
    cleaned_sentences = []
    for sentence in sentences:
        words = nltk.word_tokenize(sentence)
        filtered_words = [
            word for word in words if word.lower() not in word_list]
        cleaned_sentence = ' '.join(filtered_words)
        cleaned_sentences.append(cleaned_sentence)
    cleaned_sentences = ' '.join(cleaned_sentences)
#     print(cleaned_sentences)

#     print(positive_score)
#     print(negative_score)
    # calculate polarity score by formula
    polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
#     print(polarity_score)

    no_punct = cleaned_sentences.translate(
        str.maketrans("", "", string.punctuation))
#     print(no_punct)

    # calculate length of words in text
    words = no_punct.split()
    num_words = len(words)
#     print(num_words)

    subjectivity_score = (positive_score + negative_score) / \
        (num_words + 0.000001)
#     print(subjectivity_score)

    # calculate length of sentences
    sentences = nltk.sent_tokenize(cleaned_sentences)
    num_sentences = len(sentences)
#     print(num_sentences)

    # checking if data is empty or not in case the URL is invalid
    if num_sentences > 0 and num_words > 0:
        Average_Sentence_Length = num_words / num_sentences
    #     print(Average_Sentence_Length)
    else:
        Average_Sentence_Length = 0

    # checking for complex words
    threshold = 3
    words = cleaned_sentences.split()
    num_complex_words = 0

    for word in words:
        syllable_count = syllables.estimate(word)
        if syllable_count >= threshold:
            num_complex_words += 1

#     print(num_complex_words

    if num_complex_words > 0 and num_words > 0:
        Percentage_Complex_words = num_complex_words / num_words
    #     print(Percentage_Complex_words)
    else:
        Percentage_Complex_words = 0

    Fog_index = 0.4 * (Average_Sentence_Length + Percentage_Complex_words)
#     print(Fog_index)

    pronoun_count = re.compile(r'\b(I|we|ours|my|mine|(?-i:us))\b', re.I)
    pronouns = pronoun_count.findall(cleaned_sentences)
#     print(len(pronouns))

    words = nltk.word_tokenize(no_punct)

    total_length = sum(len(word) for word in words)

    if total_length > 0 and len(words) > 0:
        average_word_length = total_length / len(words)
    else:
        average_word_length = 0

#     print(average_word_length)
    df_output.loc[i, ['POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE', 'SUBJECTIVITY SCORE', 'AVG SENTENCE LENGTH', 'PERCENTAGE OF COMPLEX WORDS', 'FOG INDEX', 'AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT', 'WORD COUNT', 'SYLLABLE PER WORD', 'PERSONAL PRONOUNS',
                      'AVG WORD LENGTH']] = [positive_score, negative_score, polarity_score, subjectivity_score, Average_Sentence_Length, Percentage_Complex_words, Fog_index, Average_Sentence_Length, num_complex_words, num_words, syllable_count, len(pronouns), average_word_length]
    i += 1


In [ ]:
# checking to see output is as desired
print(df_output)

In [ ]:
# saving all values to output CSV file
df_output.to_csv('output.csv', index=False)